Classify CSV Data 

- Calculate LAB conversion from RGB
- Determine Monk Category based on LAB Values

Return CSV File with filled columns

In [10]:
import csv
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_objects import LabColor  
from colormath.color_diff import delta_e_cie1976 
from colormath.color_diff import delta_e_cie1976  
import pandas as pd
import math
from tqdm import tqdm
import numpy as np
from skimage.color import lab2rgb, deltaE_ciede2000


In [11]:
# Define MST reference LAB values
MST_HEX = { 
    1: '#f6ede4',  
    2: '#f3e7db',
    3: '#f7ead0',
    4: '#eadaba',
    5: '#d7bd96',
    6: '#a07e56',
    7: '#825c43',
    8: '#604134',
    9: '#3a312a',
    10: '#292420'
}  

In [12]:
# Define MST reference LAB values
MST_REF = { 
    1: LabColor(lab_l=94.211, lab_a=1.503, lab_b=5.422),  
    2: LabColor(lab_l=92.275, lab_a=2.061, lab_b=7.28), 
    3: LabColor(lab_l=93.091, lab_a=0.216, lab_b=14.205),
    4: LabColor (lab_l=87.573, lab_a=0.459, lab_b=17.748),
    5: LabColor(lab_l=77.902, lab_a=3.471, lab_b=23.136),
    6: LabColor(lab_l=55.142, lab_a=7.783, lab_b=26.74),
    7: LabColor(lab_l=42.47, lab_a=12.325, lab_b=20.53),
    8: LabColor(lab_l=30.678, lab_a=11.667, lab_b=13.335),
    9: LabColor(lab_l=21.069, lab_a=2.69, lab_b=5.964),
    10: LabColor(lab_l=14.61, lab_a=1.482, lab_b=3.525)
}  

In [18]:


def delta_e(l1, a1, b1, l2, a2, b2):
    """Calculate CIE76 color difference metric"""
    return math.sqrt((l1 - l2)**2 + (a1 - a2)**2 + (b1 - b2)**2)

# def classify_mst(L, a, b):
#     """Classify a single row to MST level"""
#     min_dist = float('inf')
#     closest = 1
    
#     for level, lab_color in MST_REF.items():
#         # Access LabColor attributes directly
#         d = delta_e(L, a, b,
#                     lab_color.lab_l,
#                     lab_color.lab_a,
#                     lab_color.lab_b)
#         if d < min_dist:
#             min_dist = d
#             closest = level
#     return closest

def classify_mst(L, a, b):
    min_de = float('inf')
    closest = 1
    
    for level, lab_color in MST_REF.items():
        lab_values = [lab_color.lab_l, lab_color.lab_a, lab_color.lab_b]
        
        current_de = deltaE_ciede2000([[L, a, b]], [lab_values]).item()
        
        if current_de < min_de:
            min_de = current_de
            closest = level
    
    return closest


def rgb_to_lab(r, g, b):
    """
    Converts RGB color values to CIELAB color space.

    Args:
        r: Red value (0-255).
        g: Green value (0-255).
        b: Blue value (0-255).

    Returns:
        A tuple containing L*, a*, and b* values of the CIELAB color space.
    """
    # Normalize RGB values
    r /= 255
    g /= 255
    b /= 255

    # Create an sRGB color object
    rgb_color = sRGBColor(r, g, b)

    # Convert to Lab color space
    lab_color = convert_color(rgb_color, LabColor)

    return round(lab_color.lab_l, 3), round(lab_color.lab_a, 3), round(lab_color.lab_b, 3)

def process_csv(input_filename, output_filename):
    """
    Reads a CSV file with RGB/HEX values and fills in L*, a*, b*, and monk_category columns.

    Args:
        input_filename: The input CSV file containing RGB/HEX values.
        output_filename: The output CSV file with updated L*, a*, b*, and monk_category columns.
    """
    
    # Load the CSV file into a DataFrame
    try:
        df = pd.read_csv(input_filename)
        print(f"Successfully loaded {len(df)} rows from {input_filename}.")
        
        # Ensure required columns exist
        required_columns = ['cielab_id',
                            'shade_id', 'hex', 'red', 'green', 'blue']
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"Missing required column: {col}")
        
        # Calculate L*, a*, b* from RGB values
        print("Converting RGB to LAB...")
        df[['L', 'a', 'b']] = df.apply(
            lambda row: pd.Series(rgb_to_lab(row['red'], row['green'], row['blue'])), axis=1)

        # Classify each row into monk_category based on LAB values
        print("Classifying rows into monk_category...")
        df['monk_category'] = df.apply(
            lambda row: classify_mst(row['L'], row['a'], row['b']), axis=1)

        # Save the updated DataFrame to a new CSV file
        df.to_csv(output_filename, index=False)
        print(f"Processing complete! Results saved to {output_filename}.")
    
    except FileNotFoundError:
        print(f"Error: File '{input_filename}' not found.")
    
    except Exception as e:
        print(f"An error occurred during processing: {e}")


In [19]:
process_csv("cielab.csv", "output.csv")

Successfully loaded 9847 rows from cielab.csv.
Converting RGB to LAB...
Classifying rows into monk_category...
Processing complete! Results saved to output.csv.
